In [3]:
import ee
import pandas as pd
import geemap
import solara
from geemap import create_download_link


In [4]:
# Define your script as a function that takes the input parameters as arguments
def run_script(startDate, endDate, s1, s2, l8, l9, export):
    # Initialize Earth Engine
    ee.Initialize()

    # Define the function to convert images in a collection to a list of dates
    def get_dates(collection, name):
        """Generates a chart of the availability of images in a collection.
        Args:
            collection: The Earth Engine image collection.
        Returns:
            A dictionary of the availability of images in the collection.
        """
        # Get the list of all available dates.
        date_range = collection.reduceColumns(ee.Reducer.toList(), ["system:time_start"])
        date_range = date_range.values().get(0)
        # Convert the list of dates to a list of strings.
        date_range = ee.List(date_range).map(lambda n: ee.Date(n).format("YYYY-MM-dd"))
        # Create a frequency histogram of the available dates.
        availability_dict = date_range.reduce(ee.Reducer.frequencyHistogram())
        # Convert the frequency histogram to a dictionary.
        availability_dict = ee.Dictionary(availability_dict).getInfo()
        for value in availability_dict:
            availability_dict[value] = name
        return availability_dict

    all_dicts = []
    # Create if statements to check if the user wants to include a satellite
    if s1:
        # Function to get dates for Sentinel-1
        def get_sentinel1_dates():
            sentinel1_collection = ee.ImageCollection("COPERNICUS/S1_GRD")\
                                    .filterDate(startDate, endDate)\
                                        .filterBounds(aoi)\
                                        .select('VV')
            return get_dates(sentinel1_collection, 'Sentinel-1')
        # Get dates for Sentinel-1
        sentinel1_data = get_sentinel1_dates()
        all_dicts.append(sentinel1_data)
    if s2:
        # Function to get dates for Sentinel-2 surface reflectance
        def get_sentinel2_dates():
            sentinel2_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
                                    .filterDate(startDate, endDate)\
                                        .filterBounds(aoi)\
                                            .select('B1')
            return get_dates(sentinel2_collection, 'Sentinel-2')
        # Get dates for Sentinel-2
        sentinel2_data = get_sentinel2_dates()
        all_dicts.append(sentinel2_data)
    if l8:
        # Function to get dates for Landsat-8
        def get_landsat8_dates():
            landsat8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
                                    .filterDate(startDate, endDate)\
                                        .filterBounds(aoi)\
                                            .select('SR_B1')
            return get_dates(landsat8_collection, 'Landsat-8')
        # Get dates for Landsat-8
        landsat8_data = get_landsat8_dates()
        all_dicts.append(landsat8_data)
    if l9:
        # Function to get dates for Landsat-9
        def get_landsat9_dates():
            landsat9_collection = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")\
                                    .filterDate(startDate, endDate)\
                                        .filterBounds(aoi)
            return get_dates(landsat9_collection, 'Landsat-9')
        # Get dates for Landsat-9
        landsat9_data = get_landsat9_dates()
        all_dicts.append(landsat9_data)
    # Create an empty dictionary to hold the combined data
    combined_dict = {}
    # Iterate over each dictionary in the list and add its items to the combined_dict
    for curr_dict in all_dicts:
        for date, code in curr_dict.items():
            # Check if the date is already in the combined_dict
            if date not in combined_dict:
                combined_dict[date] = code
    # If you want to convert it back to a dictionary and sort by date, you can do this:
    sorted_combined_dict = dict(sorted(combined_dict.items()))
    # Convert the dictionary to a Pandas DataFrame
    df = pd.DataFrame(list(sorted_combined_dict.items()), columns=['Date', 'Satellite'])

    # Plot using geemap folium map
    map = geemap.Map(center=[37.7749,-122.4194], zoom=10)
    map.add_ee_layer(ee.Image().paint(aoi, 0, 2), {}, 'AOI')
    map.add_legend(legend_title="Satellites")
    for satellite in df.continent.unique():
        # Get the subset of the DataFrame that corresponds to the current satellite
        subset = df[df['Satellite'] == satellite]
        # Convert the dates to ee.Date objects
        dates = subset['Date'].apply(ee.Date)
        # Create a feature collection with a point for each date
        features = dates.apply(lambda date: ee.Feature(ee.Geometry.Point([0, 0]), {'Date': date.format()}))
        feature_collection = ee.FeatureCollection(features.tolist())
        # Add the feature collection as a layer to the map
        map.add_ee_layer(feature_collection, {'color': satelliteColors[satellite], 'pointSize': 5}, satellite)
    # Add a download link for the data
    if export:
        link = create_download_link(df, filename='satellite_availability.csv')
        display(link)
    # Display the map
    map.add_child(geemap.LayerControl())
    map.set_center(37.7749,-122.4194)
    return map

In [5]:
from flask import Flask, render_template, request
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/run_script', methods=['POST'])
def run_script():
    # Get the input parameters from the form
    startDate = request.form['startDate']
    endDate = request.form['endDate']
    s1 = request.form.get('s1') == 'on'
    s2 = request.form.get('s2') == 'on'
    l8 = request.form.get('l8') == 'on'
    l9 = request.form.get('l9') == 'on'
    export = request.form.get('export') == 'on'
    # Call the script with the input parameters
    map = script.run_script(startDate, endDate, s1, s2, l8, l9, export)
    # Convert the map to HTML and send it as the response
    return map.to_html()

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
